In [201]:
import pickle
from tqdm import tqdm
import pandas as pd
import json


import os 

os.chdir('c:/Users/Domen/IronHack/01_projects/IH_final_project_data-jobs')
print(os.getcwd())

from src.myModules import my_NLP as myNLP

%load_ext autoreload
%autoreload 2

c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [202]:
print(os.getcwd())
with open("config/config.json", 'r') as f:
    config = json.load(f)
    
NLP_data_path = config['NLP_data_path']

print(NLP_data_path)


nlp_df_input = 'NLP_title'

c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs
data/NLP_data/


## Test Code - NLP

load cleaned dataframe

In [203]:
with open(f"{NLP_data_path}{nlp_df_input}.pkl", "rb") as file:
    df = pickle.load(file)


In [204]:
df.columns

Index(['city', 'company', 'cp1_title', 'cp2_title', 'employment_type',
       'first_match', 'id', 'industries', 'job_description', 'job_function',
       'job_level', 'keyword', 'language', 'new_job_title', 'posting_date',
       'scraping_date', 'seniority_level', 'title', 'url'],
      dtype='object')

In [205]:
print(df.columns)
nlp_df = df[df['first_match']==True]

nlp_df = nlp_df[[ 'id', 'company', 'title','new_job_title', 'job_description',  'language', 'url']]

Index(['city', 'company', 'cp1_title', 'cp2_title', 'employment_type',
       'first_match', 'id', 'industries', 'job_description', 'job_function',
       'job_level', 'keyword', 'language', 'new_job_title', 'posting_date',
       'scraping_date', 'seniority_level', 'title', 'url'],
      dtype='object')


In [206]:
keywords = ['data_analyst', 'analytics_engineer','bi_analyst', 'ml_engineer', 'business_analyst', 'business_intelligence_analyst', 'data_scientist']

In [207]:
nlp_df = nlp_df[nlp_df['new_job_title'].apply(lambda job_title: any(skill in keywords for skill in job_title))]

In [208]:
nlp_df.fillna("", inplace=True)

#### Language detection

In [209]:
tricky_re_skills = ['r', 'scala', 'sas', 'gcp']

In [210]:
skills = ['python', 'sql', 'gitlab',' r ','bi-bool','r shiny', 'power bi', 'visual analytics' 'cognos bi','aws', 'spark', 'azure', 'tableau', 'java', 'excel', 'hadoop', 'scala', 
           'snowflake', 'kafka', 'nosql', 'databricks', ' git ', 'redshift', 'airflow', 'oracle', 'sap', 'ad-hoc', 
           'sql server', 'docker', 'kubernetes', 'power_bi', 'c++', 'numpy', 'dplyr', 'matplotlib', 
           'seaborn', 'ggplot', 'hypothesis testing', 'regression analysis', 'predictive analysis', 'bayesian statistics', 
           'scikit', 'tensorflow', 'pytorch', 'xgboost', 'deep learning', 'hadoop', 'bigquery', 'airflow', 'nltk', 'spacy', 
           'nlp', 'aws', 'azure', 'looker', 'ms-office', ' ml ', ' nlp ', ' pandas', 'e2e']

First round of skill extraction from text

In [211]:
tqdm.pandas(desc="Processing")
nlp_df['job_description'] = nlp_df['job_description'].progress_apply(lambda x: x.lower().strip()) # lower job description text
nlp_df = nlp_df.progress_apply(lambda row: myNLP.check_skills(row, skills), axis=1) # use check_skill function to extract skills from text which matches words in skills list


Processing: 100%|██████████| 1687/1687 [00:04<00:00, 383.62it/s]


Investigate german and english text to identify further skills to extract 

In [212]:
nlp_df.head(10)

,company,id,job_description,language,new_job_title,skills,title,url
1,Almedia,3719620327,almedia helps companies grow by promoting thei...,en,"[data_analyst, bi_analyst]","[sql, sql, python, excel]",Data Analyst - Business Intelligence,https://www.linkedin.com/jobs/search?keywords=...
3,Ultramarin,3672508925,about the positionwe are looking for a top-tie...,en,[data_analyst],"[excel, sql, python, pandas, matplotlib]",Financial Data Analyst (m/f/d),https://www.linkedin.com/jobs/search?keywords=...
4,Les Lunes,3695466781,your missionas a business intelligence analyst...,en,[bi_analyst],"[tableau, power bi, excel, sql, excel]",Business Intelligence Analyst (f/m/d),https://www.linkedin.com/jobs/search?keywords=...
5,Orange Quarter,3585738180,orange quarter are working with an exciting st...,en,[data_analyst],[e2e],Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
6,Orange Quarter,3585734564,orange quarter are currently looking for a dat...,en,[data_analyst],"[sql, python, snowflake, bigquery]",Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
7,Enter,3704916810,build the future with enter.our mission is to ...,en,[bi_analyst],"[sql, python, snowflake]",BI Analyst (f/m/d),https://www.linkedin.com/jobs/search?keywords=...
8,Orange Quarter,3585734595,we are currently working with an emerging saas...,en,[data_analyst],"[e2e, sql, python, r ]",Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
10,kevin.,3687813018,kevin. is different. while everyone was talkin...,en,[data_analyst],"[sql, tableau, power bi, looker, sap]",Product Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
14,CBTW,3706371923,positive thinking company ist teil unseres wel...,de,[data_analyst],"[tableau, power bi, r ]",Data Analyst (m/w/d),https://www.linkedin.com/jobs/search?keywords=...
17,Qualitas Energy Deutschland GmbH,3701510210,powering change.seit mehr als 16 jahren setzen...,de,[data_analyst],[python],Data Analyst (w/m/d),https://www.linkedin.com/jobs/search?keywords=...


In [213]:
filtered_view = nlp_df[nlp_df['new_job_title'].apply(lambda job_title: any(skill in ['data_analyst'] for skill in job_title))] # filter dataset on specific job title
filtered_view.head(10)

,company,id,job_description,language,new_job_title,skills,title,url
1,Almedia,3719620327,almedia helps companies grow by promoting thei...,en,"[data_analyst, bi_analyst]","[sql, sql, python, excel]",Data Analyst - Business Intelligence,https://www.linkedin.com/jobs/search?keywords=...
3,Ultramarin,3672508925,about the positionwe are looking for a top-tie...,en,[data_analyst],"[excel, sql, python, pandas, matplotlib]",Financial Data Analyst (m/f/d),https://www.linkedin.com/jobs/search?keywords=...
5,Orange Quarter,3585738180,orange quarter are working with an exciting st...,en,[data_analyst],[e2e],Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
6,Orange Quarter,3585734564,orange quarter are currently looking for a dat...,en,[data_analyst],"[sql, python, snowflake, bigquery]",Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
8,Orange Quarter,3585734595,we are currently working with an emerging saas...,en,[data_analyst],"[e2e, sql, python, r ]",Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
10,kevin.,3687813018,kevin. is different. while everyone was talkin...,en,[data_analyst],"[sql, tableau, power bi, looker, sap]",Product Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
14,CBTW,3706371923,positive thinking company ist teil unseres wel...,de,[data_analyst],"[tableau, power bi, r ]",Data Analyst (m/w/d),https://www.linkedin.com/jobs/search?keywords=...
17,Qualitas Energy Deutschland GmbH,3701510210,powering change.seit mehr als 16 jahren setzen...,de,[data_analyst],[python],Data Analyst (w/m/d),https://www.linkedin.com/jobs/search?keywords=...
18,Talentschmiede Unternehmensberatung AG,3582381366,"as a cross-industry it management consultancy,...",en,[data_analyst],"[excel, oracle, sql]",Young Professional Client Data Analyst (m/f/d) ←,https://www.linkedin.com/jobs/search?keywords=...
20,Digital Charging Solutions,3725401052,your missionas a crucial part of the business ...,en,[data_analyst],"[scala, sql, python, tableau]",Business Process & Data Analyst (f/m/d),https://www.linkedin.com/jobs/search?keywords=...


In [214]:
df_ger = nlp_df[nlp_df['language'] == 'de']
df_eng = nlp_df[nlp_df['language'] == 'en']

In [215]:
df_ger.columns

Index(['company', 'id', 'job_description', 'language', 'new_job_title',
       'skills', 'title', 'url'],
      dtype='object')

In [216]:
df_ger = df_ger.progress_apply(lambda row: myNLP.check_german_style(row), axis=1)

Processing:  43%|████▎     | 578/1350 [00:00<00:00, 2854.85it/s]

company                                                         CBTW
id                                                        3706371923
job_description    positive thinking company ist teil unseres wel...
language                                                          de
new_job_title                                         [data_analyst]
skills                                      [tableau, power bi,  r ]
title                                           Data Analyst (m/w/d)
url                https://www.linkedin.com/jobs/search?keywords=...
Name: 14, dtype: object
company                             Qualitas Energy Deutschland GmbH
id                                                        3701510210
job_description    powering change.seit mehr als 16 jahren setzen...
language                                                          de
new_job_title                                         [data_analyst]
skills                                                      [python]
title     

Processing: 100%|██████████| 1350/1350 [00:00<00:00, 2874.44it/s]

company                           SLA Software Logistik Artland GmbH
id                                                        3713711424
job_description    über uns\r seit über 28 jahren entwickeln wir ...
language                                                          de
new_job_title                         [data_analyst, data_architect]
skills                                                [sql, tableau]
title                          Data Analyst / Data Architect (m/w/d)
url                https://www.linkedin.com/jobs/search?keywords=...
Name: 10481, dtype: object
company                                               Bankpower GmbH
id                                                        3716916089
job_description    als bankpower besetzen wir seit 1998 erfolgrei...
language                                                          de
new_job_title                                         [data_analyst]
skills                                                       [excel]
title  

In [217]:
df_eng['nlp_job_description'] = df_eng['job_description'].progress_apply(myNLP.clean_up)
df_ger['nlp_job_description'] = df_ger['job_description'].progress_apply(myNLP.stem_and_lemmatize_german) # lemmatize strings
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.tokenize) # tokenize
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.remove_stopwords_german) #remove stopwords from
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.re_blob) # reblob
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(lambda x: x.lower().strip())


Processing: 100%|██████████| 336/336 [00:00<00:00, 6593.40it/s]
C:\Users\Domen\AppData\Local\Temp\ipykernel_142068\2840816831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng['nlp_job_description'] = df_eng['job_description'].progress_apply(myNLP.clean_up)


KeyError: 'job_description'

In [ ]:
df_ger['nlp_job_description'].head(10)

In [ ]:
myNLP.word_cloud(df_ger, 'nlp_job_description')

english dataset

In [ ]:
df_eng['nlp_job_description'] = df_eng['job_description'].progress_apply(myNLP.clean_up) # clean dataset
df_eng['nlp_job_description'] = df_eng['job_description'].progress_apply(myNLP.stem_and_lemmatize_german) # lemmatize strings
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(myNLP.tokenize) # tokenize
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(myNLP.remove_stopwords_german) #remove stopwords from
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(myNLP.re_blob) # reblob
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(lambda x: x.lower().strip())

In [ ]:
display(df_eng['nlp_job_description'].head(10))

In [ ]:
myNLP.word_cloud(df_eng, 'nlp_job_description')